In [1]:
import os
import pickle
import mediapipe as mp
import cv2
import matplotlib.pyplot as plt
import numpy as np
from sklearn.preprocessing import LabelEncoder
import pandas as pd

In [4]:
DATA_DIR = 'C:/Users/terbo/Desktop/final_data_npy'
actions = []
for dir_ in os.listdir(DATA_DIR):
    actions.append(dir_)
actions 

['Act',
 'Alhamdullah',
 'all',
 'Baba',
 'Basis',
 'because',
 'Boy',
 'Brave',
 'calls',
 'calm',
 'College',
 'Confused',
 'conversation',
 'Crying',
 'daily',
 'danger',
 'disagree with',
 'drinks',
 'Eats',
 'effort',
 'Egypt',
 'Enters',
 'Excelent',
 'explanation',
 'Fasting',
 'Female',
 'First',
 'For Us',
 'fuel',
 'Gift',
 'Girl',
 'Glass',
 'good bye',
 'government',
 'Happy',
 'hates',
 'hears',
 'Help',
 'Here You Are',
 'how_are_u',
 'Humanity',
 'hungry',
 'I',
 'ignorance',
 'immediately',
 'Important',
 'Intelligent',
 'Last',
 'leader',
 'Liar',
 'Loves',
 'male',
 'Mama',
 'memory',
 'model',
 'mostly',
 'motive',
 'Muslim',
 'must',
 'my home land',
 'no',
 'nonsense',
 'obvious',
 'Old',
 'Palestine',
 'prevent',
 'ready',
 'rejection',
 'Right',
 'selects',
 'shut up',
 'Sing',
 'sleeps',
 'smells',
 'Somkes',
 'Spoon',
 'Summer',
 'symposium',
 'Tea',
 'teacher',
 'terrifying',
 'Thanks',
 'time',
 'to boycott',
 'to cheer',
 'to go',
 'to live',
 'to spread',
 

In [5]:
label_map = {label:num for num, label in enumerate(actions)}
label_map

{'Act': 0,
 'Alhamdullah': 1,
 'all': 2,
 'Baba': 3,
 'Basis': 4,
 'because': 5,
 'Boy': 6,
 'Brave': 7,
 'calls': 8,
 'calm': 9,
 'College': 10,
 'Confused': 11,
 'conversation': 12,
 'Crying': 13,
 'daily': 14,
 'danger': 15,
 'disagree with': 16,
 'drinks': 17,
 'Eats': 18,
 'effort': 19,
 'Egypt': 20,
 'Enters': 21,
 'Excelent': 22,
 'explanation': 23,
 'Fasting': 24,
 'Female': 25,
 'First': 26,
 'For Us': 27,
 'fuel': 28,
 'Gift': 29,
 'Girl': 30,
 'Glass': 31,
 'good bye': 32,
 'government': 33,
 'Happy': 34,
 'hates': 35,
 'hears': 36,
 'Help': 37,
 'Here You Are': 38,
 'how_are_u': 39,
 'Humanity': 40,
 'hungry': 41,
 'I': 42,
 'ignorance': 43,
 'immediately': 44,
 'Important': 45,
 'Intelligent': 46,
 'Last': 47,
 'leader': 48,
 'Liar': 49,
 'Loves': 50,
 'male': 51,
 'Mama': 52,
 'memory': 53,
 'model': 54,
 'mostly': 55,
 'motive': 56,
 'Muslim': 57,
 'must': 58,
 'my home land': 59,
 'no': 60,
 'nonsense': 61,
 'obvious': 62,
 'Old': 63,
 'Palestine': 64,
 'prevent': 65,
 

In [6]:
temp_labels = np.array(actions)
temp_labels.shape[0]

100

In [7]:
sequence_length = 30
sequences, labels = [], []
for action in actions:
    for sequence in np.array(os.listdir(os.path.join(DATA_DIR, action))).astype(int):
        window = []
        for frame_num in range(1,30,2):
            res = np.load(os.path.join(DATA_DIR, action, str(sequence), "{}.npy".format(frame_num))) 
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])

In [8]:
X = np.array(sequences)
print(X.shape)

(26100, 15, 84)


In [9]:
import tensorflow.keras.utils as tf_utils
y = tf_utils.to_categorical(labels).astype(int)
print(y.shape)

(26100, 100)


In [10]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
# X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

In [11]:
import tensorflow as tf
class StopOnAccuracy(tf.keras.callbacks.Callback):
    def __init__(self, target_acc):
        self.target_acc = target_acc

    def on_epoch_end(self, epoch, logs=None):
        if logs['categorical_accuracy'] >= self.target_acc:
            print("\nReached target accuracy, stopping training...")
            self.model.stop_training = True

In [12]:
from tensorflow import keras
from keras.utils import plot_model
from tensorflow.keras import regularizers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, GRU

model = Sequential()

model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(15,84)))
model.add(Dropout(0.4)) 
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(Dropout(0.2)) 
model.add(LSTM(64, return_sequences=False, activation='relu'))

# model.add(GRU(64, return_sequences=True, activation='relu', input_shape=(15,84)))
# model.add(Dropout(0.4)) 
# model.add(GRU(128, return_sequences=True, activation='relu'))
# model.add(Dropout(0.2)) 
# model.add(GRU(64, return_sequences=False, activation='relu'))

model.add(Dense(64, activation='relu', kernel_regularizer=regularizers.l2(0.001))) 
model.add(Dense(32, activation='relu', kernel_regularizer=regularizers.l2(0.001))) 
model.add(Dense(temp_labels.shape[0], activation='softmax'))

model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])
stop_on_acc = StopOnAccuracy(0.99)  # Stop if validation accuracy reaches 99%
# history = model.fit(X_train, y_train,
#                     validation_data=(X_val, y_val),
#                     epochs=2000,
#                     callbacks=[stop_on_acc])
model.fit(X_train, y_train, epochs=700 ,callbacks=[stop_on_acc])
model.summary()

# Create a graphical representation of the model architecture
keras.utils.plot_model(model, to_file='model_architecture.png', show_shapes=True)


c:\Users\terbo\anaconda3\envs\sign_language_env\Lib\site-packages\keras\src\layers\rnn\rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/700
653/653 ━━━━━━━━━━━━━━━━━━━━ 19s 21ms/step - categorical_accuracy: 0.0748 - loss: 3.6131
Epoch 2/700
653/653 ━━━━━━━━━━━━━━━━━━━━ 14s 22ms/step - categorical_accuracy: 0.2142 - loss: 2.5482
Epoch 3/700
653/653 ━━━━━━━━━━━━━━━━━━━━ 13s 20ms/step - categorical_accuracy: 0.3124 - loss: 2.1466
Epoch 4/700
653/653 ━━━━━━━━━━━━━━━━━━━━ 13s 21ms/step - categorical_accuracy: 0.4109 - loss: 1.8371
Epoch 5/700
653/653 ━━━━━━━━━━━━━━━━━━━━ 13s 20ms/step - categorical_accuracy: 0.5207 - loss: 1.4757
Epoch 6/700
653/653 ━━━━━━━━━━━━━━━━━━━━ 13s 20ms/step - categorical_accuracy: 0.6030 - loss: 1.2122
Epoch 7/700
653/653 ━━━━━━━━━━━━━━━━━━━━ 13s 20ms/step - categorical_accuracy: 0.6618 - loss: 1.0489
Epoch 8/700
653/653 ━━━━━━━━━━━━━━━━━━━━ 13s 20ms/step - categorical_accuracy: 0.7120 - loss: 0.9010
Epoch 9/700
653/653 ━━━━━━━━━━━━━━━━━━━━ 13s 20ms/step - categorical_accuracy: 0.7493 - loss: 0.7954
Epoch 10/700
653/653 ━━━━━━━━━━━━━━━━━━━━ 13s 20ms/step - categorical_accuracy: 0.7747 - lo

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                     │ (None, 15, 64)         │        38,144 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 15, 64)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ (None, 15, 128)        │        98,816 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 15, 128)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_2 (LSTM)                   │ (None, 64)             │        49,408 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 64)             │         4,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 100)            │         3,300 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 587,726 (2.24 MB)

 Trainable params: 195,908 (765.27 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 391,818 (1.49 MB)

You must install graphviz (see instructions at https://graphviz.gitlab.io/download/) for `plot_model` to work.


In [13]:
model.save('action3.h5')
model.load_weights('action3.h5')

In [14]:
from keras.models import load_model

# Path to your saved model file (.h5 format)

# Load the model
# modelGRU = load_model("action2.h5")
modelLSTM =  load_model("action3.h5")

In [ ]:
# from sklearn.metrics import multilabel_confusion_matrix, accuracy_score
# from sklearn.metrics import precision_recall_fscore_support

# yhat = modelGRU.predict(X_test)
# ytrue = np.argmax(y_test, axis=1).tolist()
# yhat = np.argmax(yhat, axis=1).tolist()
# multilabel_confusion_matrix(ytrue, yhat)


164/164 [==============================] - 2s 7ms/step


array([[[5180,    0],
        [   0,   40]],

       [[5151,    0],
        [   0,   69]],

       [[5179,    0],
        [   0,   41]],

       [[5176,    2],
        [   0,   42]],

       [[5175,    0],
        [   0,   45]],

       [[5171,    0],
        [   0,   49]],

       [[5176,    1],
        [   0,   43]],

       [[5170,    0],
        [   0,   50]],

       [[5160,    0],
        [   1,   59]],

       [[5171,    0],
        [   0,   49]],

       [[5148,    0],
        [   0,   72]],

       [[5175,    0],
        [   0,   45]],

       [[5172,    0],
        [   0,   48]],

       [[5160,    0],
        [   0,   60]],

       [[5153,    0],
        [   0,   67]],

       [[5174,    0],
        [   0,   46]],

       [[5165,    0],
        [   1,   54]],

       [[5171,    0],
        [   1,   48]],

       [[5165,    0],
        [   1,   54]],

       [[5173,    1],
        [   1,   45]],

       [[5174,    0],
        [   0,   46]],

       [[5159,    0],
        [   

In [ ]:
# from sklearn.metrics import recall_score

# # Assuming y_true and y_pred are your true and predicted labels respectively
# # Calculate recall
# recall = recall_score(ytrue, yhat, average='macro')  # Change 'macro' to 'micro' or 'weighted' if needed
# print("Recall:", recall)

Recall: 0.9967551234800647


In [ ]:
# from sklearn.metrics import precision_score

# # Assuming y_true and y_pred are your true and predicted labels respectively
# # Calculate precision
# precision = precision_score(ytrue, yhat, average='macro')  # Change 'macro' to 'micro' or 'weighted' if needed
# print("Precision:", precision)

Precision: 0.9969855023432665


In [28]:
# from sklearn.metrics import f1_score

# # Assuming y_true and y_pred are your true and predicted labels respectively
# # Calculate F1 score
# f1 = f1_score(ytrue, yhat, average='macro')  # Change 'macro' to 'micro' or 'weighted' if needed
# print("F1 Score:", f1)

F1 Score: 0.996795360577872


In [ ]:
# GRU_ACC= accuracy_score(ytrue, yhat)
# print("Accuracy", GRU_ACC)

Accuracy 0.9971264367816092


In [15]:
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score
from sklearn.metrics import precision_recall_fscore_support

yhat = modelLSTM.predict(X_test)
ytrue = np.argmax(y_test, axis=1).tolist()
yhat = np.argmax(yhat, axis=1).tolist()
multilabel_confusion_matrix(ytrue, yhat)


164/164 ━━━━━━━━━━━━━━━━━━━━ 2s 10ms/step


array([[[5180,    0],
        [   1,   39]],

       [[5151,    0],
        [   0,   69]],

       [[5176,    3],
        [   1,   40]],

       [[5178,    0],
        [   0,   42]],

       [[5175,    0],
        [   0,   45]],

       [[5171,    0],
        [   0,   49]],

       [[5177,    0],
        [   1,   42]],

       [[5170,    0],
        [   0,   50]],

       [[5160,    0],
        [   2,   58]],

       [[5171,    0],
        [   0,   49]],

       [[5146,    2],
        [   0,   72]],

       [[5170,    5],
        [   0,   45]],

       [[5171,    1],
        [   0,   48]],

       [[5159,    1],
        [   0,   60]],

       [[5153,    0],
        [   0,   67]],

       [[5174,    0],
        [   0,   46]],

       [[5165,    0],
        [   0,   55]],

       [[5169,    2],
        [   1,   48]],

       [[5165,    0],
        [   1,   54]],

       [[5174,    0],
        [   1,   45]],

       [[5174,    0],
        [   1,   45]],

       [[5158,    1],
        [   

In [16]:
from sklearn.metrics import recall_score

# Assuming y_true and y_pred are your true and predicted labels respectively
# Calculate recall
recall = recall_score(ytrue, yhat, average='macro')  # Change 'macro' to 'micro' or 'weighted' if needed
print("Recall:", recall)

Recall: 0.9878946843899962


In [17]:
from sklearn.metrics import precision_score

# Assuming y_true and y_pred are your true and predicted labels respectively
# Calculate precision
precision = precision_score(ytrue, yhat, average='macro')  # Change 'macro' to 'micro' or 'weighted' if needed
print("Precision:", precision)

Precision: 0.9876503610225201


In [18]:
from sklearn.metrics import f1_score

# Assuming y_true and y_pred are your true and predicted labels respectively
# Calculate F1 score
f1 = f1_score(ytrue, yhat, average='macro')  # Change 'macro' to 'micro' or 'weighted' if needed
print("F1 Score:", f1)

F1 Score: 0.9875062835804083


In [19]:
LSTM_ACC= accuracy_score(ytrue, yhat)
print("Accuracy", LSTM_ACC)

Accuracy 0.9877394636015325
